<a href="https://colab.research.google.com/github/Devikasree099/FMML-register/blob/main/Copy_of_FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1st Answer

When adjusting the percentage of the validation set:

1. **Increase Validation Set**:
   - **More Data for Validation**: Gives a better estimate of model performance.
   - **Reduced Training Data**: Can slightly decrease model accuracy due to less training data.
   
2. **Decrease Validation Set**:
   - **More Data for Training**: Can improve model accuracy by providing more data for learning.
   - **Less Reliable Validation**: May give less accurate estimates of performance due to smaller validation samples.

The balance between training and validation data affects both model learning and the reliability of performance estimates.
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Split data with different validation sizes
train_sizes = [0.9, 0.8, 0.7]  # Percentages of data used for training
accuracies = []

for train_size in train_sizes:
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=train_size, random_state=42)
    
    # Train model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    
    # Validate model
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append((train_size, accuracy))

# Output results
for train_size, accuracy in accuracies:
    print(f"Train size: {train_size*100}%, Validation Accuracy: {accuracy*100:.2f}%")

2nd Answer

The size of the training and validation sets directly impacts how accurately the validation set can predict test set performance:

Larger Training Set: Leads to better model learning and generalization, making the validation accuracy a more reliable predictor of test accuracy.
Larger Validation Set: Provides more stable and reliable accuracy estimates, reducing the variance and making it a closer reflection of test accuracy.
Smaller Training Set: Results in underfitting, reducing the model’s ability to generalize, which may cause the validation accuracy to poorly reflect test accuracy.
Smaller Validation Set: Increases variability and reduces the reliability of the validation accuracy, making it less predictive of test set performance.
In summary, a balanced split between training and validation data is crucial for accurate test performance prediction.
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target

# Define a function to evaluate model performance on different splits
def evaluate_model(train_size):
    # Split the dataset into training + validation and test sets
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Split the remaining data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, train_size=train_size, random_state=42)
    
    # Train a Random Forest model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate on validation set
    val_pred = model.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_pred)
    
    # Evaluate on test set
    test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_pred)
    
    print(f"Train Size: {train_size * 100:.0f}%, Validation Accuracy: {val_accuracy * 100:.2f}%, Test Accuracy: {test_accuracy * 100:.2f}%")

# Test different train-validation splits
for train_size in [0.5, 0.6, 0.7, 0.8, 0.9]:
    evaluate_model(train_size)

3rd Answer

A common practice is to reserve 10-20% of your dataset for the validation set. This range typically strikes a good balance between having enough data to provide a reliable estimate of model performance and retaining sufficient data for training.

Considerations:
10-20% Validation Set:

Pros: Provides a stable estimate of model performance with a good balance of training data.
Cons: For very large datasets, even smaller percentages (e.g., 5-10%) can be sufficient.
Dataset Size:

Small Datasets: For smaller datasets, you might opt for the higher end of this range (15-20%) to ensure a robust validation estimate while still having enough data for training.
Large Datasets: For larger datasets, a smaller percentage (e.g., 10%) might be adequate, as the absolute number of validation samples will still be substantial.
Alternatives:
Cross-Validation: For small datasets or when fine-tuning is crucial, k-fold cross-validation (e.g., 5-fold or 10-fold) can provide a more comprehensive evaluation by using different subsets for validation and training across multiple folds, maximizing data use and reducing variability.
Ultimately, the choice depends on the dataset size and specific requirements of your model evaluation.
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load a sample dataset
data = load_iris()
X, y = data.data, data.target

# Function to evaluate the impact of different train-validation splits
def evaluate_splits(train_size, validation_size, test_size=0.2):
    # Split the data into train + validation and test
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    # Split the train_val set into actual train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, train_size=train_size, test_size=validation_size, random_state=42)
    
    # Train the model on the training set
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate on the validation and test sets
    val_accuracy = accuracy_score(y_val, model.predict(X_val))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    
    print(f"Training Size: {len(X_train)}, Validation Size: {len(X_val)}, Test Size: {len(X_test)}")
    print(f"Validation Accuracy: {val_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}\n")
    
# Experiment with different splits
evaluate_splits(train_size=0.5, validation_size=0.3)  # Example 1: Smaller training set
evaluate_splits(train_size=0.7, validation_size=0.1)  # Example 2: Larger training set
evaluate_splits(train_size=0.6, validation_size=0.2)  # Example 3: Balanced split

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1st Answer

Yes, averaging the validation accuracy across multiple splits generally gives more consistent and reliable results. This approach is commonly used in cross-validation, a technique where the dataset is divided into multiple subsets (or "folds"), and the model is trained and validated on different combinations of these subsets. Here's why averaging across splits is beneficial:

Reduces Variance: A single validation split might give you an accuracy that is highly dependent on how the data was split. By averaging across multiple splits, you reduce the influence of any particular split, leading to a more stable estimate of the model's performance.

Better Generalization: Averaging across splits gives you a more accurate measure of how the model is expected to perform on unseen data. It simulates the model's performance across different scenarios, leading to a more generalizable estimate.

Mitigates Overfitting: If a model overfits to a particular training set, this might not be reflected in a single validation split but will be caught when multiple splits are averaged. This makes it a better indicator of true performance.

Improved Robustness: Since each split tests the model on a different subset of data, averaging the results increases robustness against the variability inherent in the data.

In summary, averaging validation accuracy across multiple splits generally leads to a more consistent and reliable assessment of a model's performance.


Certainly! Below is an example in Python using scikit-learn to perform cross-validation and average the validation accuracy across multiple splits.



from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Average the validation accuracy across the splits
average_accuracy = cv_scores.mean()

# Print individual accuracies and the average accuracy
print(f'Cross-validation accuracies: {cv_scores}')
print(f'Average validation accuracy: {average_accuracy:.4f}')
Explanation:
cross_val_score: This function splits the data into cv (in this case 5) folds, trains the model on cv-1 folds, and validates it on the remaining fold. It repeats this process for all folds.
cv_scores: An array containing the accuracy for each fold.
average_accuracy: The mean accuracy across all folds, giving a more consistent and reliable estimate of the model's performance.
Output:
The output will include the validation accuracies for each fold and the average accuracy across all folds.

This approach can be applied to any model and dataset, and increasing the number of folds (cv) can lead to even more consistent results.

2nd Answer

Averaging validation accuracy across multiple splits provides a more accurate estimate of test accuracy by:

1. **Reducing Overfitting Risk**: It prevents the model from overfitting to a specific train-validation split.
  
2. **Capturing Data Variability**: It accounts for the natural variability in the dataset, leading to a more representative assessment of model performance.

3. **Mitigating Randomness**: Reduces the influence of random factors that might skew results in a single split.

4. **Ensuring Generalization**: It better reflects how the model will perform on truly unseen data, as the evaluation considers multiple possible scenarios.

Overall, this approach gives a more robust and accurate prediction of test accuracy, enhancing confidence in the model's generalization ability.
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Calculate the mean and standard deviation of validation accuracy
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

# Output results
print(f"Mean Validation Accuracy: {mean_cv_score*100:.2f}%")
print(f"Validation Accuracy Std Dev: {std_cv_score*100:.2f}%")

3rd Answer

When estimating test accuracy in machine learning, the number of iterations typically refers to the number of times a model is trained and evaluated on different subsets of data. This can occur in various contexts, such as cross-validation, bootstrapping, or iterative model training methods like stochastic gradient descent.

Effect of the Number of Iterations on the Estimate:
Increased Stability: More iterations generally lead to a more stable and reliable estimate of test accuracy. With more iterations, the variability in the results due to different training-test splits or random factors is averaged out, providing a more accurate reflection of the model's true performance.

Reduced Variance: As the number of iterations increases, the variance of the accuracy estimate tends to decrease. This means that the accuracy estimate becomes less sensitive to the specific data splits or random seeds used, giving a more consistent measure of the model's performance.

Better Generalization: More iterations can also help in better understanding the model's generalization ability, as it is tested on various data splits or augmented data points, leading to a more robust estimate of its real-world performance.

Diminishing Returns: After a certain point, increasing the number of iterations provides diminishing returns. The estimate of test accuracy converges to a stable value, and additional iterations only marginally improve the estimate, if at all.

Summary:
Yes, generally, we get a better estimate of test accuracy with a higher number of iterations, as it reduces variability and provides a more robust measure of the model's performance. However, after a certain point, the benefit of additional iterations becomes minimal.


 import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

# Load the dataset
data = load_iris()
X, Y = data.data, data.target

# Initialize the model
model = RandomForestClassifier()

# Number of iterations
n_iterations = 100

# Store the cross-validation scores
cv_scores = []

# Perform cross-validation multiple times
for i in range(n_iterations):
    # Define the k-fold cross-validation
    kfold = KFold(n_splits=5, shuffle=True, random_state=i)
    
    # Perform cross-validation and store the mean score
    scores = cross_val_score(model, X, Y, cv=kfold)
    cv_scores.append(np.mean(scores))

# Convert list to a numpy array
cv_scores = np.array(cv_scores)

# Calculate mean and standard deviation of the scores
mean_accuracy = np.mean(cv_scores)
std_accuracy = np.std(cv_scores)

print(f'Mean Accuracy after {n_iterations} iterations: {mean_accuracy:.4f}')
print(f'Standard Deviation of Accuracy after {n_iterations} iterations: {std_accuracy:.4f}')

4th Answer

Increasing the number of iterations with a very small training dataset can lead to overfitting, where the model becomes too specialized in the limited training data and does not generalize well to new, unseen data. Similarly, a very small validation dataset does not provide a reliable measure of the model's performance, so increasing the number of iterations won't improve the validation quality or the model's generalization ability.

Instead of simply increasing iterations, it's often better to use strategies like data augmentation to artificially expand the training dataset, transfer learning from pre-trained models, or cross-validation techniques to make the most of the available data.
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers

# Load and preprocess the data (replace 'data/train' and 'data/val' with actual paths)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    'data/val',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Load a pre-trained model (transfer learning)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model with additional layers on top of the pre-trained model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

# Train the model with a small dataset
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,  # Use a reasonable number of epochs to avoid overfitting
    validation_data=validation_generator,
    validation_steps=50
)

# Evaluate the model on the validation data
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {validation_accuracy}')

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.